In [1]:
## Importation
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
from datetime import datetime
import re
import psycopg2 as pg
from psycopg2 import Error
import configparser
abcsite = 'https://www.abc.net.au'


In [5]:
c = configparser.ConfigParser()
c.read('config/1kindtask.ini')
c.sections()
c['DEFAULT']['log_formatter']


InterpolationSyntaxError: '%' must be followed by '%' or '(', found: '%\\(asctime\\)s \\%\\(levelname\\)s \\%\\(message\\)s'

In [18]:
try:
    # Connect to an existing database
    connection = pg.connect(user = 'robins',
    password = 'CIoXUd7cu6fyCk85ckHH',
    host = 'postgredb.c4ddacllbiwy.ap-southeast-2.rds.amazonaws.com',
    port = '5432',
    database = 'postgres')

    # Create a cursor to perform database operations
    cursor = connection.cursor()
    # Print PostgreSQL details
    print("PostgreSQL server information")
    print(connection.get_dsn_parameters(), "\n")
    # Executing a SQL query
    cursor.execute('set search_path to \'1kindtask\'')
    cursor.execute("SELECT * from ini")
    # Fetch result
    record = cursor.fetchone()
    print("You are connected to - ", record, "\n")

except (Exception, Error) as error:
    print("Error while connecting to PostgreSQL", error)
finally:
    if (connection):
        cursor.close()
        connection.close()
        print("PostgreSQL connection is closed")


PostgreSQL server information
{'user': 'robins', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': 'postgredb.c4ddacllbiwy.ap-southeast-2.rds.amazonaws.com', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'} 

You are connected to -  None 

PostgreSQL connection is closed


In [3]:
## request for site mainpage and category pages

abcsite = 'https://www.abc.net.au'
init = requests.get(abcsite+'/news/')
f = open("news/abc/mainpage"+str(datetime.now()).replace(':','-')+".html", "w")
f.write(init.text)
f.close()

s = BeautifulSoup(init.text,'lxml')
abc_cate=s.find_all('a',class_='_3cShj _2p6Xq')
abc_cates_href =[]
abc_cates = []
for i in abc_cate:
    if i['href'] not in abc_cates_href:
        abc_cates.append({'name':i.text,'href':i['href'],'mainpage':requests.get(abcsite+i['href'])})
##print (abc_cates)

In [25]:
href = '/news/2022-07-22/new-york-washington-mayors-ask-biden-help-asylum-seekers/101262182'
error_log_path = 'log/abc/errors/'
try:
    newspage = requests.get(abcsite+href)
    newspage.raise_for_status() # To make sure that the website is accessible when it return code 200
except requests.HTTPError as inst:
    print(inst)
    f = open(error_log_path+datetime.now().strftime('%Y-%m-%d')+'.txt', "a")
    f.write(datetime.now().strftime('%Y-%m-%d %H:%M:%S')+'    '+str(inst)+'\n')
    f.close()
    # Access error, return null
else:
    f = open("news/abc/newspage/"+'test'+str(datetime.now()).replace(':','-')+".html", "w")
    f.write(newspage.text)
    f.close()

In [5]:
## get site categories
for i in abc_cates:
    currentsoup = BeautifulSoup(i['mainpage'].text,'lxml')
    current_cate_article = currentsoup.find_all('a',class_='_2VA9J u0kBv _3CtDL _1_pW8 _3Fvo9 VjkbJ')



In [6]:
testsoup = BeautifulSoup(abc_cates[0]['mainpage'].text,'lxml')
cate_news = testsoup.find_all('a',class_='_2VA9J u0kBv _3CtDL _1_pW8 _3Fvo9 VjkbJ')
abc_cates[0]



{'name': 'Just In', 'href': '/news/justin/', 'mainpage': <Response [200]>}

In [15]:
# %%
## Importation
import sys
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os
import logging
from datetime import datetime
import re



# %%
abcsite = 'https://www.abc.net.au'
error_log_path = '/log/abc/errors/'
## request for site mainpage and category pages
def fetch_abc_news_main_page():
    try:
        mainpage = requests.get(abcsite+'/news/')
        mainpage.raise_for_status() # To make sure that the website is accessible when it return code 200
    except Exception as inst:
        print(inst)
        f = open(error_log_path+datetime.now().strftime('%Y-%m-%d')+'.txt', "a")
        f.write(datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')+'     Fetching ABC NEWS mainpage receiving '+mainpage.status_code)
        f.close()
        return None
        # Access error, return null
    
    f = open("news/abc/mainpage"+str(datetime.now()).replace(':','-')+".html", "w")
    f.write(mainpage.text)
    f.close()
    return mainpage
##print (abc_cates)

# %%
## get ABC News mainpage news and get ABC News categories
def analyse_main_page(mainpage):
    soup = BeautifulSoup(mainpage.text,'lxml')
    abc_cates=soup.find_all('a',class_='_3cShj _2p6Xq')
    abc_cates_href =[]
    abc_cates_label = []
    for i in abc_cates:
        if i['href'] not in abc_cates_href:
            abc_cates_label.append(i.text)
            abc_cates_href.append(i['href'])
    abc_cates_df = pd.DataFrame({
        'label' : abc_cates_label,
        'href' : abc_cates_href
    })
    abc_mainpage_news_teaser = []
    abc_mainpage_news_category = []
    abc_mainpage_news_href = []
    abc_mainpage_news_date = []
    abc_mainpage_news_lists = [
        s.find_all('a',class_='_3pM2c u0kBv _3CtDL _1_pW8 _3Fvo9 VjkbJ'),
        s.find_all('a',class_='_2VA9J u0kBv _3CtDL _1_pW8 _3Fvo9 VjkbJ'),
        s.find_all('a',class_='_37gvg u0kBv _3CtDL _1_pW8 _3Fvo9 VjkbJ')
    ]
    find_category = lambda x: x if x is None else x.group(1)
    for i in abc_mainpage_news_lists:
        for j in i:
            abc_mainpage_news_teaser.append(j.text)
            abc_mainpage_news_href.append(j['href'])
            abc_mainpage_news_date.append(re.search('\\d{4}-\\d{2}-\\d{2}',i['href']).group())
            abc_mainpage_news_category.append(find_category(re.search('/news/(.*)/\\d{4}-\\d{2}-\\d{2}',i['href'])))


    abc_mainpage_news_df = pd.DataFrame({
        'teaser' : abc_mainpage_news_teaser,
        'href' : abc_mainpage_news_href,
        'date' : abc_mainpage_news_date,
        'category' : abc_mainpage_news_category
    })
    return abc_cates_df, abc_mainpage_news_df









    ##print (abc_cates)   
    for i in abc_cates:
        currentsoup = BeautifulSoup(i['mainpage'].text,'lxml')
        current_cate_articles = currentsoup.find_all('a',class_='_2VA9J u0kBv _3CtDL _1_pW8 _3Fvo9 VjkbJ')











# %%
if __name__== '__Main__':
    error_count = 0
    #fetch ABC News Mainpage
    mainpage = fetch_abc_news_main_page()
    while mainpage is None:
        if error_count ==5:
            f = open(error_log_path+datetime.now().strftime('%Y-%m-%d')+'.txt', "a")
            f.write(datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')+'     Fetching ABC News 5 times failed, exit app')
            f.close()
            sys.exit('5 times failed to fetch ABC NEWS mainpage, exiting app')
        os.sleep(5000)
        error_count+=1
        mainpage = fetch_abc_news_main_page()
    
    #generate dataframes from mainpage
    try:
        abc_cates_df, abc_mainpage_news_df = analyse_main_page(mainpage)
    except Exception as e:
        print(e.with_traceback)
        f = open("log/abc/errors/"+datetime.now().strftime('%Y-%m-%d')+'.txt', "a")
        f.write(datetime.now().strftime('%Y-%m-%dT%H:%M:%SZ')+'     Analysing ABC News mainpage failed, exit app')
        f.close()
        sys.exit('Analysing ABC News mainpage failed, exit app')

    #fetch news from categories
    for i in abc_cates_df:
        print(i)



    #fetch news contents
    s = BeautifulSoup(mainpage.text,'lxml')
    abc_cate=s.find_all('a',class_='_3cShj _2p6Xq')
    abc_cates_href =[]
    abc_cates = []
    for i in abc_cate:
        if i['href'] not in abc_cates_href:
            abc_cates.append({'name':i.text,'href':i['href'],'mainpage':requests.get(abcsite+i['href'])})
    ##print (abc_cates)   
    for i in abc_cates:
        currentsoup = BeautifulSoup(i['mainpage'].text,'lxml')
        current_cate_article = currentsoup.find_all('a',class_='_2VA9J u0kBv _3CtDL _1_pW8 _3Fvo9 VjkbJ')



# %%
testsoup = BeautifulSoup(abc_cates[0]['mainpage'].text,'lxml')
cate_news = testsoup.find_all('a',class_='_2VA9J u0kBv _3CtDL _1_pW8 _3Fvo9 VjkbJ')
abc_cates[0]



# %%
newsc = requests.get(abcsite+cate_news[0]['href'])
newssoup = BeautifulSoup(newsc.text,'lxml')

# %%
news_title = newssoup.find('h1').text
news_body = newssoup.find('div', id = 'body')
news_body.find_all(class_='r6CUb u0kBv _3CtDL _1_pW8 _3Fvo9')
news_topics = []
for i in news_body.find_all(class_='r6CUb u0kBv _3CtDL _1_pW8 _3Fvo9'):
    news_topics.append({'Label':i.text,'Link':i['href']})
news_body.find_all('p' or 'a')    


# %%
newssoup1 = BeautifulSoup(requests.get('https://www.abc.net.au/news/2022-07-22/new-york-washington-mayors-ask-biden-help-asylum-seekers/101262182').text,'lxml')
news_title1 = newssoup1.find('h1').text
news_body1 = newssoup1.find('div', id = 'body')
news_body1.find_all(class_='r6CUb u0kBv _3CtDL _1_pW8 _3Fvo9')
news_body1.find_all('p' or 'a')
newssoup1.find('h1')
newssoup1.find(class_='timestamp')

# %%
news_title1 = newssoup1.find('h1').text
news_body1 = newssoup1.find('div', id = 'body')
#news_author1 = news_body1.strong

news_time1 = newssoup1.find(class_="_1EAJU _1NECW _2L258 _14LIk _3pVeq hmFfs _2F43D")['datetime']
news_date1 = news_time1[:10]                       #datetime.date(datetime.strptime(news_time1,"%Y-%m-%dT%H:%M:%S.%fZ")).strftime('%Y-%m-%d')
news_body1.find_all(class_='r6CUb u0kBv _3CtDL _1_pW8 _3Fvo9')

news_topic1 = news_body1.find(class_='_2wDgl').find_all('a')
news_topics1 = []
for i in news_topic1:
    news_topics1.append({'Label':i.text,'Link':i['href']})
news_category1 = abc_cates[0]
news_content1 = ''
for i in news_body1.find_all('p'):
    news_content1 += i.text+'\n'
newssoup1.find('h1')
newssoup1.find(class_='timestamp')

# %%
news_content1 = ''
for i in news_body1.find_all('p'):
    news_content1 += i.text+'\n'
print(news_content1)


if __name__== '__Main__':
    error_count = 0


The Democratic mayors of New York City and Washington DC are calling on the Biden administration to address what they say is a surge of migrants seeking asylum after travelling from Republican-controlled border states to their cities.
Washington Mayor Muriel Bowser and New York Mayor Eric Adams, both Democrats, have been plunged into a debate over migrants who appear at the US-Mexico border who then take up an offer of state-funded bus trips to the two larger cities after their release by federal immigration authorities. 
Mr Adams criticised the tactic that has been employed by Texas Governor Greg Abbott and Arizona Governor Doug Ducey, both Republicans. 
"The mere fact that they sent people out of their states, people who were seeking refuge in our country, then sent them away — did they deny that?" he asked.
"They ended up here because they didn't get the support there."
Mr Abbott and Mr Ducey have trumpeted the bus trips, which have been running for months.
While they have been big 

In [7]:
newsc = requests.get(abcsite+cate_news[0]['href'])
newssoup = BeautifulSoup(newsc.text,'lxml')

In [8]:
news_title = newssoup.find('h1').text
news_body = newssoup.find('div', id = 'body')
news_body.find_all(class_='r6CUb u0kBv _3CtDL _1_pW8 _3Fvo9')
news_topics = []
for i in news_body.find_all(class_='r6CUb u0kBv _3CtDL _1_pW8 _3Fvo9'):
    news_topics.append({'Label':i.text,'Link':i['href']})
news_body.find_all('p' or 'a')    


[<p class="_1HzXw">A fast-moving brush fire near Yosemite National Park has exploded in size into one of California's largest wildfires of the year, prompting evacuation orders for thousands of people and shutting off power to more than 2,000 homes and businesses.</p>,
 <p class="_1HzXw">The Oak Fire started on Friday afternoon south-west of the park near the town of Midpines in Mariposa County and by Saturday had grown to nearly 48 square kilometres, according to the California Department of Forestry and Fire Protection, or Cal Fire.</p>,
 <p class="_1HzXw">It erupted as firefighters made progress against an <a class="u0kBv _3CtDL _1_pW8 _3Fvo9" data-component="ContentLink" data-uri="coremedia://article/101264358" href="/news/2022-07-24/us-takes-emergency-action-to-save-sequoias-from-wildfires/101264358">earlier blaze that burned to the edge of a grove of giant sequoias in the southernmost part of Yosemite park</a>.</p>,
 <p class="_1HzXw">Evacuation orders were put in effect on Satur

In [9]:
newssoup1 = BeautifulSoup(requests.get('https://www.abc.net.au/news/2022-07-22/new-york-washington-mayors-ask-biden-help-asylum-seekers/101262182').text,'lxml')
news_title1 = newssoup1.find('h1').text
news_body1 = newssoup1.find('div', id = 'body')
news_body1.find_all(class_='r6CUb u0kBv _3CtDL _1_pW8 _3Fvo9')
news_body1.find_all('p' or 'a')
newssoup1.find('h1')
newssoup1.find(class_='timestamp')

In [10]:
news_title1 = newssoup1.find('h1').text
news_body1 = newssoup1.find('div', id = 'body')
#news_author1 = news_body1.strong

news_time1 = newssoup1.find(class_="_1EAJU _1NECW _2L258 _14LIk _3pVeq hmFfs _2F43D")['datetime']
news_date1 = news_time1[:10]                       #datetime.date(datetime.strptime(news_time1,"%Y-%m-%dT%H:%M:%S.%fZ")).strftime('%Y-%m-%d')
news_body1.find_all(class_='r6CUb u0kBv _3CtDL _1_pW8 _3Fvo9')

news_topic1 = news_body1.find(class_='_2wDgl').find_all('a')
news_topics1 = []
for i in news_topic1:
    news_topics1.append({'Label':i.text,'Link':i['href']})
news_category1 = abc_cates[0]
news_content1 = ''
for i in news_body1.find_all('p'):
    news_content1 += i.text+'\n'
newssoup1.find

<bound method Tag.find of <!DOCTYPE html>
<html data-react-hydrated="false" lang="en-AU">
<head>
<title data-react-helmet="true">Mayors request President Joe Biden's help amid 'unprecedented surge' of asylum seekers after Republican Governors 'sent them away' on buses  - ABC News</title>
<meta charset="utf-8" data-react-helmet="true"/>
<meta content="width=device-width, initial-scale=1.0" data-react-helmet="true" name="viewport"/>
<meta content="ie=edge" data-react-helmet="true" http-equiv="x-ua-compatible"/>
<meta content="Mayors request President Joe Biden's help amid 'unprecedented surge' of asylum seekers after Republican Governors 'sent them away' on buses  - ABC News" data-react-helmet="true" name="title"/>
<meta content="The Democratic mayors of New York City and Washington DC are calling on the Biden administration to address what they say is a surge of migrants seeking asylum after travelling from Republican-controlled border states to their cities." data-react-helmet="true" n

In [11]:
news_content1 = ''
for i in news_body1.find_all('p'):
    news_content1 += i.text+'\n'
print(news_content1)

The Democratic mayors of New York City and Washington DC are calling on the Biden administration to address what they say is a surge of migrants seeking asylum after travelling from Republican-controlled border states to their cities.
Washington Mayor Muriel Bowser and New York Mayor Eric Adams, both Democrats, have been plunged into a debate over migrants who appear at the US-Mexico border who then take up an offer of state-funded bus trips to the two larger cities after their release by federal immigration authorities. 
Mr Adams criticised the tactic that has been employed by Texas Governor Greg Abbott and Arizona Governor Doug Ducey, both Republicans. 
"The mere fact that they sent people out of their states, people who were seeking refuge in our country, then sent them away — did they deny that?" he asked.
"They ended up here because they didn't get the support there."
Mr Abbott and Mr Ducey have trumpeted the bus trips, which have been running for months.
While they have been big 